In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import os

os.environ['OPENBLAS_NUM_THREADS']='1'
 
articles_df = pd.read_csv('D:/书籍资料整理/kaggle/CI&T/shared_articles.csv')
interactions_df = pd.read_csv('D:/书籍资料整理/kaggle/CI&T/users_interactions.csv')
articles_df.drop(['authorUserAgent', 'authorRegion', 'authorCountry'], axis=1, inplace=True)
interactions_df.drop(['userAgent', 'userRegion', 'userCountry'], axis=1, inplace=True)
articles_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [2]:
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.drop('eventType', axis=1, inplace=True)
df = pd.merge(interactions_df[['contentId','personId', 'eventType']], articles_df[['contentId', 'title']], how = 'inner', on = 'contentId')
df.head()

,contentId,personId,eventType,title
0,-3499919498720038879,-8845298781299428018,VIEW,Hiri wants to fix the workplace email problem
1,-3499919498720038879,-8845298781299428018,VIEW,Hiri wants to fix the workplace email problem
2,-3499919498720038879,-108842214936804958,VIEW,Hiri wants to fix the workplace email problem
3,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem
4,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem


In [3]:
df['eventType'].value_counts()

VIEW               61043
LIKE                5745
BOOKMARK            2463
COMMENT CREATED     1611
FOLLOW              1407
Name: eventType, dtype: int64

VIEW: 查看，表示用户点击过该篇文章<br>
LIKE:  点赞，表示用户喜好该篇文章。<br>
BOOKMARK: 加入书签，用户已将该文章加入书签，以便将来查看。这是用户对该文章表现出感兴趣的迹象<br>
COMMENT CREATED: 创建评论，说明用户被这篇文章所吸引，同时希望表达自己的观点。<br>
FOLLOW: 关注，用户关注了文章的作者，这应该说明用户对这位作者的文章非常感兴趣<br>

In [4]:

event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 3.0, 
   'FOLLOW': 4.0,
   'COMMENT CREATED': 5.0,  
}
 
df['eventStrength'] = df['eventType'].apply(lambda x: event_type_strength[x])
df.sample(10)

,contentId,personId,eventType,title,eventStrength
69946,-14569272361926584,9210530975708218054,BOOKMARK,Java 8 Streams - A Deeper Approach About Perfo...,3.0
48478,-1633984990770981161,-6944500707172804068,LIKE,UX ou UI?,2.0
43283,1881534532776527237,301435144665447655,BOOKMARK,9 Non-Threatening Leadership Strategies for Women,3.0
20658,833256912680066502,-1443636648652872475,LIKE,"Rumor: ""Android VR"" Will Launch At I/O With St...",2.0
43011,-2423593292146522678,6464364285286199002,VIEW,Services | RaRe Technologies,1.0
45204,-7584037043735285197,2542290381109225938,VIEW,Seguradores serão mentores de start-ups seleci...,1.0
42224,4105873627900556540,1623838599684589103,VIEW,Sua comunidade de Teste.,1.0
22686,1933229167501870037,-709287718034731589,VIEW,Perfect Menu for Mobile Apps - UX Planet,1.0
7896,6369367631770966886,-492179582639058400,LIKE,Como a Netflix influencia o que você vai assis...,2.0
41818,-2447632164766022033,4254153380739593270,VIEW,O que o GitHub tem a nos dizer sobre os estere...,1.0


In [5]:
df = df.drop_duplicates()
grouped_df = df.groupby(['personId', 'contentId', 'title']).sum().reset_index()
grouped_df.sample(10)

,personId,contentId,title,eventStrength
9076,-4384580909851654688,5445999759138709405,Bradesco lança serviço inédito para estimular ...,1.0
12795,-2936612463070256553,3306277069425849869,Google segue Microsoft e lança ferramenta anal...,1.0
33167,5000549443219971803,-8370744479086515302,Mobile banking conquista o coração dos brasile...,1.0
17133,-1367069781843778997,-6444494220871855741,Novo padrão de rede cabeada pode oferecer até ...,1.0
7313,-5636811822913690686,2581138407738454418,10 Modern Software Over-Engineering Mistakes,1.0
6843,-5868110530814399805,5816734694927565572,Bot Design Patterns,1.0
8825,-4585796377251906117,3643465492984939400,Production Ready Jenkins 2.0 in Two Steps with...,1.0
40551,9148269800512008413,4061646345741687197,O homofóbico dentro de cada um,12.0
23450,1296936597887557880,-6255158415883847921,"Para neurociência, motivação não é fator princ...",1.0
27479,3094513233385472738,-36844712915106340,Windows 10 Anniversary SDK is bringing excitin...,1.0


In [6]:
grouped_df['title'] = grouped_df['title'].astype("category")
grouped_df['personId'] = grouped_df['personId'].astype("category")
grouped_df['contentId'] = grouped_df['contentId'].astype("category")
grouped_df['person_id'] = grouped_df['personId'].cat.codes
grouped_df['content_id'] = grouped_df['contentId'].cat.codes
grouped_df.sample(10)

,personId,contentId,title,eventStrength,person_id,content_id
27740,3217014177234377440,2081512200574495983,Chip Pro sequel to crowdsourced $9 computer se...,1.0,1280,1798
18013,-1032019229384696495,2075656428057535795,What exercise does to your bones,1.0,831,1796
31854,4254153380739593270,-7722236679806608274,"Chrome 53 Beta: Shadow DOM, PaymentRequest, an...",1.0,1390,256
4065,-7421703586506797266,-2962461122453899874,Honda revela conceito de carro com 'emoções' e...,1.0,183,1015
26611,2760160224512620259,8302949268716967655,What makes a good (Android) Software Engineer,1.0,1223,2827
13879,-2626634673110551643,6062146090334604102,Serverless Data Sync in Web Apps with Bit Torrent,10.0,682,2461
35955,6735372008307093370,2888507406903473601,Amor sem fronteiras: série de fotos retrata ca...,1.0,1627,1939
27550,3106760029136205156,358192030955601318,A nova sede da LEGO na Dinamarca é toda sobre ...,10.0,1266,1539
13817,-2626634673110551643,4575580212020245658,Conversational Banking; from branches to bots....,1.0,682,2203
39026,8433143903150367236,3689128258624052102,Deputada mais jovem da Câmara silencia o plená...,1.0,1799,2069


In [7]:
sparse_content_person = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['content_id'], grouped_df['person_id'])))
sparse_person_content = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['person_id'], grouped_df['content_id'])))
 
print(sparse_content_person.shape)
print(sparse_person_content.shape)

(2979, 1895)
(1895, 2979)


In [8]:
alpha = 15
data = (sparse_content_person * alpha).astype('double')
 
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)
model.fit(data)

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
content_id=235
n_similar=10

#获取用户矩阵
person_vecs = model.user_factors
#获取内容矩阵
content_vecs = model.item_factors
#计算内容的向量的范数
content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1))
#计算指定的content_id 与其他所有文章的相似度
scores = content_vecs.dot(content_vecs[content_id]) / content_norms
#获取相似度最大的10篇文章
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
#组成content_id和title的元组
similar = sorted(zip(top_idx, scores[top_idx] / content_norms[content_id]), key=lambda x: -x[1])
 
print(person_vecs.shape)
print(content_vecs.shape)

#下面我们展示这10篇最相似的文章title:
for content in similar:
    idx, score = content
    print(grouped_df.title.loc[grouped_df.content_id == idx].iloc[0],"|",score)

(1895, 20)
(2979, 20)
Artificial intelligence is hard to see | 0.9999999
Machine Learning Is No Longer Just for Experts | 0.7807914
Clarifying the uses of artificial intelligence in the enterprise | 0.75835997
ProtoHack, hackathon criado no Vale do Silício, desembarca em São Paulo para sua primeira edição no espaço Engenho Maker - Startupi | 0.7402122
Qualcomm's deep learning SDK will mean more AI on your smartphone | 0.6598807
Google just open sourced something called 'Parsey McParseface,' and it could change AI forever | 0.6552687
The Pulse of a City - Uber Engineering Blog | 0.6362824
Machine Learning Unconference | 0.6286892
My Top 9 Favorite Python Deep Learning Libraries - PyImageSearch | 0.622092
What readers think about computer-generated texts | 0.62133956


In [12]:
idx2=model.recommend(50,sparse_person_content,N=10)
idx2=pd.DataFrame(idx2,columns=['content_id','score'])
idx2.head()
temp=grouped_df[['content_id','title']].drop_duplicates()
temp=temp.merge(idx2,how='inner',on='content_id')
temp.sort_values(by='score',inplace=True,ascending=False)
temp

,content_id,title,score
3,1094,Custo do Erro - Cinco motivos para investir em...,1.722936
8,1908,'The Simpsons' celebrates 600 episodes with a ...,1.203732
0,845,Former Google career coach shares a visual tri...,1.124764
7,624,BDD Best Practices and Guidelines - Testing Ex...,1.072531
5,1449,"Novo workaholic trabalha, pratica esportes e t...",1.069973
1,2968,15 minutos sobre Docker,1.047490
9,2034,15 Interesting JavaScript and CSS Libraries fo...,1.017476
2,902,Do You Suffer From Deployment Anxiety? - DZone...,0.999666
6,2065,Discutindo Devops na Prática,0.988838
4,1881,10 Modern Software Over-Engineering Mistakes,0.983226
